<a href="https://colab.research.google.com/github/gragbag/NestedFunctionSyntaxChecker/blob/main/NestedFunctionSyntaxChecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class PDA:
  def __init__(self):
        self.stack = ['Z']  # Start with the initial symbol Z on the stack
        self.state = 'q0'
        self.keywords = {'int', 'float', 'void', '=', ',', '(', ')', '{', '}', ';'}
        self.transitions = {
          ('q0', None, 'Z'): [('q1', 'Program Z')],

          # Program structure
          ('q1', None, 'Program'): [('q1', 'FuncDecl FuncCall')],

          # Function Declaration: (ReturnType FuncName ( ParamList ) Block)
          ('q1', None, 'FuncDecl'): [('q1', 'ReturnType FuncName ( ParamList ) Block')],

          # ReturnType is int, float, or void
          ('q1', 'int', 'ReturnType'): [('q1', None)],
          ('q1', 'float', 'ReturnType'): [('q1', None)],
          ('q1', 'void', 'ReturnType'): [('q1', None)],

          # Parameter list for functions (either a single parameter or more)
          ('q1', None, 'ParamList'): [('q1', 'Param'),
                                      ('q1', 'Param , ParamList'),
                                      ('q1', None)],

          ('q1', 'int', 'Param'): [('q1', 'var')],
          ('q1', 'float', 'Param'): [('q1', 'var')],

          # Function name
          ('q1', 'var', 'FuncName'): [('q1', None)],

          # Block starts with '{', ends with '}'
          ('q1', '{', 'Block'): [('q1', 'StmtList }')],

          # Statements can be assignments or function calls
          ('q1', None, 'StmtList'): [('q1', 'Stmt'), ('q1', 'Stmt StmtList')],

          # A single statement can either be an assignment or a function call
          ('q1', None, 'Stmt'): [('q1', 'Assign'), ('q1', 'FuncCall')],

          # Assignment statement
          ('q1', None, 'Assign'): [('q1', 'var = Expr ;')],

          # Expressions can be variables, numbers, or function calls
          ('q1', 'var', 'Expr'): [('q1', None)],
          ('q1', 'num', 'Expr'): [('q1', None)],
          ('q1', None, 'Expr'): [('q1', 'FuncCall')],

          # Function calls have a function name followed by an argument list
          ('q1', 'var', 'FuncCall'): [('q1', '( ArgList ) ;'), ('q1', '( ArgList )')],

          # Argument list (may have no arguments, one, or more arguments)
          ('q1', None, 'ArgList'): [('q1', 'Expr'), ('q1', 'Expr , ArgList'), ('q1', None)],

          # Handle terminal symbols (variables, function names, types, operators)
          ('q1', 'int', 'int'): [('q1', None)],
          ('q1', 'float', 'float'): [('q1', None)],
          ('q1', 'void', 'void'): [('q1', None)],
          ('q1', 'var', 'var'): [('q1', None)],
          ('q1', '(', '('): [('q1', None)],
          ('q1', ')', ')'): [('q1', None)],
          ('q1', '{', '{'): [('q1', None)],
          ('q1', '}', '}'): [('q1', None)],
          ('q1', ';', ';'): [('q1', None)],
          ('q1', '=', '='): [('q1', None)],
          ('q1', ',', ','): [('q1', None)],

          # Accepting state: when the input is fully processed, and stack is back to initial symbol
          ('q1', None, 'Z'): [('qf', None)]
      }

  def get_possible_transitions(self, symbol, top_of_stack):
    return self.transitions.get((self.state, symbol, top_of_stack))

  def process_input(self, input_program):
    tokens = input_program.split()

    # Recursive backtracking helper function
    def backtrack(index):
      if index == len(tokens):
        # If we've processed all tokens and stack is at the initial state, return True
        return self.stack[-1] == 'Z'

      top_of_stack = self.stack.pop()
      curr_state = self.state
      curr_stack = self.stack.copy()

      token = tokens[index]

      print(curr_stack, ", Top:  ", top_of_stack ,", Token: ", token, end = " ")

      #Include as token
      possible_transitions = self.get_possible_transitions(token, top_of_stack)

      if possible_transitions:
        for next_state, variables in possible_transitions:
          self.state = next_state

          if variables is not None:
            for var in reversed(variables.split()):
              self.stack.append(var)

          print("   <--- Using Token: ")
          if backtrack(index + 1):
            return True


          self.state = curr_state
          self.stack = curr_stack

      #Include as variable
      if token not in self.keywords:
        possible_transitions = self.get_possible_transitions('var', top_of_stack)
        if possible_transitions:
          for next_state, variables in possible_transitions:
            self.state = next_state

            if variables is not None:
              for var in reversed(variables.split()):
                self.stack.append(var)

            print("   <--- Using Var: ")
            if backtrack(index + 1):
              return True

            self.state = curr_state
            self.stack = curr_stack

      #Include as number
      if is_number(token):
        possible_transitions = self.get_possible_transitions('num', top_of_stack)
        if possible_transitions:
          for next_state, variables in possible_transitions:
            self.state = next_state

            if variables is not None:
              for var in reversed(variables.split()):
                self.stack.append(var)

            print("   <--- Using Num: ")
            if backtrack(index + 1):
              return True

            self.state = curr_state
            self.stack = curr_stack

      #Do not include token
      possible_transitions = self.get_possible_transitions(None, top_of_stack)
      if possible_transitions:
        for next_state, variables in possible_transitions:
          self.state = next_state

          if variables is not None:
            for var in reversed(variables.split()):
              self.stack.append(var)

            print("    <--- Using None: ")
            if backtrack(index):
              return True
          # else:
          #   if backtrack(index + 1):
          #     return True

          self.state = curr_state
          self.stack = curr_stack

      return False


    return backtrack(0)

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def main():
  pda = PDA()
  program = "int f ( int a , int b ) { a = f ( 5 , 23 ) ; b = 10 ; } f ( g ( 15 ) , 12 ) ; "

  if pda.process_input(program):
    print("The program syntax is valid!")
  else:
    print("The program syntax is invalid!")

main()

[] , Top:   Z , Token:  int     <--- Using None: 
['Z'] , Top:   Program , Token:  int     <--- Using None: 
['Z', 'FuncCall'] , Top:   FuncDecl , Token:  int     <--- Using None: 
['Z', 'FuncCall', 'Block', ')', 'ParamList', '(', 'FuncName'] , Top:   ReturnType , Token:  int    <--- Using Token: 
['Z', 'FuncCall', 'Block', ')', 'ParamList', '('] , Top:   FuncName , Token:  f    <--- Using Var: 
['Z', 'FuncCall', 'Block', ')', 'ParamList'] , Top:   ( , Token:  (    <--- Using Token: 
['Z', 'FuncCall', 'Block', ')'] , Top:   ParamList , Token:  int     <--- Using None: 
['Z', 'FuncCall', 'Block', ')'] , Top:   Param , Token:  int    <--- Using Token: 
['Z', 'FuncCall', 'Block', ')'] , Top:   var , Token:  a    <--- Using Var: 
['Z', 'FuncCall', 'Block'] , Top:   ) , Token:  ,     <--- Using None: 
['Z', 'FuncCall', 'Block', ')', 'ParamList', ','] , Top:   Param , Token:  int    <--- Using Token: 
['Z', 'FuncCall', 'Block', ')', 'ParamList', ','] , Top:   var , Token:  a    <--- Using Va